In [1]:
import numpy as np
from astropy.table import Table, vstack

In [2]:
path = './../catalogs/'
VU_GC = Table.read(path+'VUGC_pPXF_2017_v2.fits')
VD_GC = Table.read(path+'VDGC_pPXF_2017_v2.fits')

In [3]:
columns_to_change= ['ZERR', 'ZERR_ne', 'ZERR_pe', 'ZHEL', 'ZOBS', 'ZOBS_ne', 'ZOBS_pe']

for col in columns_to_change:
    VD_GC[col] = VD_GC[col] * 3e+5
    
    VD_GC.rename_column(col,'V' + col[1:])
    
VU_GC['KECKID'] = '---'
VU_GC['ZQUAL'] = -101010
VU_GC['ZSPECNUM'] = -101010



In [4]:
test_VDGC = VD_GC[VD_GC['ZCONF']==1]
test_VDGC = test_VDGC[test_VDGC['GCSAT']== 'Y']
Host = VU_GC[((VU_GC['HOST']== 'VLSB-B') | (VU_GC['HOST']== 'VLSB-D') | (VU_GC['HOST']=='VCC0615')) & (VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='Y')]
Not_host = VU_GC[VU_GC['ZCONF']==1]
Not_host = Not_host[Not_host['GCSAT']=='Y']
Not_host = Not_host[((Not_host['HOST'] != 'VLSB-B') & (Not_host['HOST'] != 'VLSB-D') & (Not_host['HOST'] != 'VCC0615'))]
test_VDGC= vstack([test_VDGC, Not_host])
Host.remove_rows(np.nonzero(Not_host)[0])

In [5]:
# test_VDGC.remove_rows(np.nonzero((test_VDGC['VHEL']>-300)  & (test_VDGC['VHEL'] < 300)))
#  Host.remove_rows(np.nonzero((Host['VHEL'] > -300) & (Host['VHEL'] < 300)))

In [6]:
Host

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,MASKNAME,SLITNUM,YLOW,YHIGH,SPEC1DNAME,KECKID,ZQUAL,ZSPECNUM
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,int16,int16,int16,bytes40,str3,int64,int64
---,GCXL,Y,VLSB-B,187.04613,12.724055,52.14,9.228409,7.88,8.58,1.0,104.23,7.88,8.58,42.49,4.41,3.94,9.6,4.28,vugc1,10,12,22,spec1d.vugc1.010.GCXL012.fits.gz,---,-101010,-101010
---,GCXF,Y,VLSB-B,187.04088,12.735666,-73.27,48.365,39.31,57.42,1.0,-63.67,39.31,57.42,7.27,17.61,183.55,9.6,0.81,vugc1,19,15,23,spec1d.vugc1.019.GCXF011.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17159,13.580055,1006.92,8.453587,4.7,7.19,1.0,987.41,4.7,7.19,-28.51,5.59,6.43,9.0,6.17,vugc5,1,4,7,spec1d.vugc5.001.GCXL001.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18745,13.564,1057.16,10.3895645,7.36,7.03,1.0,1052.7,7.36,7.03,-13.46,7.44,7.55,9.0,6.33,vugc5,2,24,32,spec1d.vugc5.002.GCXL002.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17741,13.550445,1026.31,4.880886,3.9,4.49,1.0,1028.8,3.9,4.49,-6.51,2.44,2.55,9.0,11.62,vugc5,3,55,63,spec1d.vugc5.003.GCXL003.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18413,13.556723,1056.22,12.89066,15.56,8.47,1.0,1056.32,15.56,8.47,-8.9,4.43,4.91,9.0,2.39,vugc5,4,16,22,spec1d.vugc5.004.GCXL004.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18338,13.483723,1019.73,12.122438,7.14,5.4,1.0,1072.4,7.14,5.4,43.67,13.04,7.71,9.0,5.08,vugc5,6,72,82,spec1d.vugc5.006.GCXL007.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18124,13.522139,1016.17,2.5402215,2.32,2.21,1.0,1037.49,2.32,2.21,12.32,1.11,1.19,9.0,21.19,vugc5,7,44,52,spec1d.vugc5.007.GCXL009.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.15533,13.511806,1030.47,18.865732,9.73,7.2,1.0,1047.31,9.73,7.2,7.84,16.95,16.77,9.0,4.41,vugc5,10,66,78,spec1d.vugc5.010.GCXL013.fits.gz,---,-101010,-101010


In [7]:
VUGC_VDGC=vstack([test_VDGC,Host])

VUGC_VDGC

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40
---,GCA,Y,VCC1075,187.05533,10.299723,1795.8826,6.6943264,7.1255054,5.546625,1.0,1810.4825,7.014853,5.403738,0.00026271507,8.415822e-05,7.671974e-05,4.866667e-05,3.34,gcA10,vdgc1,11,12,3,25,31,spec1d.vdgc1.011.gcA10.fits.gz
---,GCA,Y,VCC1075,187.04483,10.286805,1843.0951,10.339908,9.879919,10.394689,1.0,1855.5137,6.974825,6.2042923,-7.271697e-06,2.2949209e-05,2.7485681e-05,4.866667e-05,5.36,gcA17,vdgc1,12,13,4,17,23,spec1d.vdgc1.012.gcA17.fits.gz
---,GCA,Y,VCC1075,187.03674,10.289222,1802.7073,6.052629,4.5716085,6.9834056,1.0,1821.8304,3.3122916,6.584555,1.50770975e-05,9.640003e-06,6.5378563e-06,4.866667e-05,7.06,gcA19,vdgc1,14,15,4,39,46,spec1d.vdgc1.014.gcA19.fits.gz
---,GCA,Y,VCC1075,187.04433,10.315306,1861.2278,2.5437276,1.4287839,1.2718638,1.0,1875.8278,0.69047767,0.23015922,0.0012339203,0.00022345458,0.00019736988,4.866667e-05,2.93,gcA20,vdgc1,15,16,1,32,36,spec1d.vdgc1.015.gcA20.fits.gz
---,GCN,Y,VCC1079,187.0423,10.3538885,1323.2156,12.560184,13.214163,11.683929,1.0,1333.0521,12.498647,9.956888,-1.5877651e-05,1.3676128e-05,1.9947132e-05,4.866667e-05,6.54,gcN120,vdgc1,26,31,1,122,128,spec1d.vdgc1.026.gcN120.fits.gz
---,GCN,Y,VCC1079,187.05162,10.367778,1338.6162,13.627155,13.83195,13.110909,1.0,1369.9778,9.316445,7.795393,5.5871988e-05,3.38234e-05,3.4890803e-05,4.866667e-05,6.97,gcN138,vdgc1,27,32,4,32,38,spec1d.vdgc1.027.gcN138.fits.gz
---,GCN,Y,VCC1079,187.05696,10.373972,1313.909,11.331903,11.525938,10.722818,1.0,1330.3203,7.775379,7.475171,6.03751e-06,2.805274e-05,2.5284158e-05,4.866667e-05,4.94,gcN145,vdgc1,29,34,4,20,26,spec1d.vdgc1.029.gcN145.fits.gz
---,GCN,Y,VCC1079,187.05429,10.377334,1358.58,6.8130116,8.182219,4.8669662,1.0,1376.2621,7.945497,4.2029076,1.0273774e-05,5.0034614e-06,7.0382025e-06,4.866667e-05,7.02,gcN147,vdgc1,30,35,4,30,36,spec1d.vdgc1.030.gcN147.fits.gz
---,GCN,Y,VCC1076,187.0515,10.524445,1650.4119,11.4403515,11.538051,10.929236,1.0,1669.485,10.947573,10.317138,1.4910315e-05,1.1407892e-05,1.12744665e-05,4.866667e-05,6.99,gcN206,vdgc1,37,44,4,114,120,spec1d.vdgc1.037.gcN206.fits.gz


In [8]:
fileOrphanDE = Table.read(path + 'orphanGCs.fits')
filekinDE = Table.read(path + 'VDGC_kinematic_prob.fits')
fileOrphanUDG = Table.read(path + 'SelectGCsTrue2.fits')
filekinUDG = Table.read(path + 'SelectGCsTrue_kinematic_prob.fits')

In [9]:
def typeandprob(file):
    probability= []
    objecttype= []
    for row in file:
        probability.append(max(row))
        vals = []
        for ind, val in enumerate(row):
            vals.append(val)
        if (vals.index(max(vals))==0):
            objecttype.append('Star')
        elif (vals.index(max(vals))==1):
            objecttype.append('ICGC')
        elif (vals.index(max(vals))==2):
            objecttype.append('M87GC')
    return probability, objecttype

In [10]:
probabilityDE, objecttypeDE = typeandprob(filekinDE)
probabilityUDG, objecttypeUDG = typeandprob(filekinUDG)


In [11]:
fileOrphanDE['Probability'] = probabilityDE
fileOrphanUDG['Probability'] = probabilityUDG
fileOrphanDE['Type'] = objecttypeDE
fileOrphanUDG['Type'] = objecttypeUDG

In [12]:
noDE = VD_GC[(VD_GC['ZCONF']==1) & (VD_GC['GCSAT']=='N') | (VD_GC['GCSAT']=='M')]
noUDG = VU_GC[(VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='N') | (VU_GC['GCSAT']=='M')]


In [13]:
noDE.sort('SPEC1DNAME')
noUDG.sort('SPEC1DNAME')
fileOrphanDE.sort('SPEC1DNAME')
fileOrphanUDG.sort('SPEC1DNAME')

focusedUDG = fileOrphanUDG['SPEC1DNAME', 'Probability', 'Type']
focusedUDG.add_row(['spec1d.vugc1.044.GCXO0504.fits.gz', -1, 'Unknown'])
focusedUDG.sort('SPEC1DNAME')

noDE['Probability'] = probabilityDE
noUDG['Probability'] = focusedUDG['Probability']
noDE['Type'] = objecttypeDE
noUDG['Type'] = focusedUDG['Type']

In [14]:
noUDG

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,MASKNAME,SLITNUM,YLOW,YHIGH,SPEC1DNAME,KECKID,ZQUAL,ZSPECNUM,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,int16,int16,int16,bytes40,str3,int64,int64,float32,str5
VCC1148,NUC,N,---,187.2422,12.661695,1417.16,0.30438462,0.21,0.24,1.0,1403.28,0.21,0.24,-23.48,0.18,0.23,9.6,151.96,vugc1,0,43,59,spec1d.vugc1.000.NUC001.fits.gz,---,-101010,-101010,0.84855264,M87GC
---,GCXL,N,---,187.0645,12.741167,-591.31,5.6009865,3.6,4.99,1.0,-544.81,3.6,4.99,36.9,3.44,3.75,9.6,7.79,vugc1,7,58,68,spec1d.vugc1.007.GCXL008.fits.gz,---,-101010,-101010,0.9999366,ICGC
---,GCXL,N,---,187.04138,12.743,-339.2,12.014878,1.14,18.9,1.0,-271.02,1.14,18.9,58.58,5.66,7.6,9.6,2.14,vugc1,9,41,51,spec1d.vugc1.009.GCXL011.fits.gz,---,-101010,-101010,0.6393685,ICGC
---,GCXL,N,---,187.26938,12.675,1265.91,10.11425,9.15,6.84,1.0,1251.6,9.15,6.84,-23.91,6.78,5.61,9.6,4.19,vugc1,11,16,26,spec1d.vugc1.011.GCXL013.fits.gz,---,-101010,-101010,0.83778,M87GC
---,GCXL,N,---,187.0158,12.7225275,1397.37,7.2989125,5.45,7.26,1.0,1452.44,5.45,7.26,45.47,3.44,3.74,9.6,6.01,vugc1,12,12,22,spec1d.vugc1.012.GCXL015.fits.gz,---,-101010,-101010,0.8498808,M87GC
---,GCXL,N,---,187.25525,12.648389,951.12,30.016117,12.48,16.38,1.0,943.01,12.48,16.38,-17.71,28.32,24.32,9.6,2.21,vugc1,14,60,70,spec1d.vugc1.014.GCXL017.fits.gz,---,-101010,-101010,0.57101935,M87GC
---,GCBL,N,---,187.07175,12.729305,190.86,12.783288,3.2,4.53,1.0,220.64,3.2,4.53,20.18,13.53,10.84,9.6,7.97,vugc1,25,11,19,spec1d.vugc1.025.GCBL001.fits.gz,---,-101010,-101010,0.9511431,Star
---,GCBL,N,---,187.27963,12.6585,-14.56,0.59682494,0.53,0.49,1.0,-35.26,0.53,0.49,-30.3,0.3,0.32,9.6,61.11,vugc1,29,48,58,spec1d.vugc1.029.GCBL007.fits.gz,---,-101010,-101010,0.98372805,Star
---,GCBL,N,---,187.24742,12.637139,-55.26,5.19,4.92,5.46,1.0,-45.66,4.92,5.46,-8.24,29.95,48.57,9.6,7.79,vugc1,30,13,23,spec1d.vugc1.030.GCBL008.fits.gz,---,-101010,-101010,0.98111,Star


In [15]:
noDE_UDG=vstack([noDE,noUDG])

noDE_UDG

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40,float32,str5
---,ALG,N,---,187.0097,10.359333,54.56785,2.5169947,1.2761084,1.2873586,1.0,117.61137,0.21014537,0.21014537,0.00016147838,4.6698975e-07,7.3384103e-07,4.866667e-05,196.03,alg42,vdgc1,0,0,-1,15,21,spec1d.vdgc1.000.alg42.fits.gz,0.98157275,Star
---,ALG,N,---,187.07608,10.3505,-210.7357,3.0194468,2.024616,2.1834822,1.0,-170.35785,1.5010384,1.6711562,8.592611e-05,1.7678897e-06,2.1348103e-06,4.866667e-05,58.89,alg115,vdgc1,1,1,-1,15,21,spec1d.vdgc1.001.alg115.fits.gz,0.90411496,Star
---,ALG,N,---,186.9995,10.459528,12.869002,2.8418825,1.7621847,1.9203647,1.0,76.86318,1.0807476,1.2108377,0.00016464724,2.0347409e-06,2.701869e-06,4.866667e-05,44.0,alg140,vdgc1,2,2,-1,15,21,spec1d.vdgc1.002.alg140.fits.gz,0.982906,Star
---,ALG,N,---,187.04004,10.45025,19.96391,2.5854344,1.3975698,1.426399,1.0,32.882748,0.46031845,0.57039464,-5.6038766e-06,1.4009692e-06,1.2675436e-06,4.866667e-05,80.71,alg164,vdgc1,3,3,-1,16,21,spec1d.vdgc1.003.alg164.fits.gz,0.9828415,Star
---,ALG,N,---,187.07025,10.325111,-2.7317898,2.612219,1.4655958,1.453245,1.0,34.784065,0.65045,0.6404431,7.6386175e-05,1.3342564e-06,1.2341872e-06,4.866667e-05,92.15,alg177,vdgc1,4,4,-1,15,20,spec1d.vdgc1.004.alg177.fits.gz,0.98282486,Star
---,ALG,N,---,187.05908,10.544111,17.422152,2.510731,1.271273,1.2664591,1.0,-29.680532,0.17011769,0.14009692,-0.00020567562,5.003461e-07,4.6698975e-07,4.866667e-05,222.86,alg191,vdgc1,5,5,-1,15,21,spec1d.vdgc1.005.alg191.fits.gz,0.9828719,Star
---,ALG,N,---,187.06075,10.386139,-53.867165,2.7268543,1.7217433,1.5956256,1.0,-18.522814,0.73050535,0.70048463,6.9147834e-05,3.1021461e-06,2.3349487e-06,4.866667e-05,73.2,alg194,vdgc1,6,6,-1,15,20,spec1d.vdgc1.006.alg194.fits.gz,0.9802294,Star
VCC1075,NUC,N,---,187.05128,10.297639,1811.7435,2.917756,1.9805114,1.9400272,1.0,1825.7831,1.4409969,1.2608724,-1.867959e-06,1.7678897e-06,2.6018e-06,4.866667e-05,28.58,nuc5,vdgc1,7,7,4,30,37,spec1d.vdgc1.007.nuc5.fits.gz,1.0,ICGC
VCC1079,NUC,N,---,187.05008,10.365334,1334.1931,3.0186136,2.089962,2.114991,1.0,1349.0332,1.5510731,1.541066,8.0055383e-07,2.101454e-06,2.435018e-06,4.866667e-05,32.52,nuc6,vdgc1,8,8,4,19,25,spec1d.vdgc1.008.nuc6.fits.gz,1.0,ICGC


In [16]:
table1=vstack([VUGC_VDGC,noDE_UDG])

table1

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40,float32,str5
---,GCA,Y,VCC1075,187.05533,10.299723,1795.8826,6.6943264,7.1255054,5.546625,1.0,1810.4825,7.014853,5.403738,0.00026271507,8.415822e-05,7.671974e-05,4.866667e-05,3.34,gcA10,vdgc1,11,12,3,25,31,spec1d.vdgc1.011.gcA10.fits.gz,--,--
---,GCA,Y,VCC1075,187.04483,10.286805,1843.0951,10.339908,9.879919,10.394689,1.0,1855.5137,6.974825,6.2042923,-7.271697e-06,2.2949209e-05,2.7485681e-05,4.866667e-05,5.36,gcA17,vdgc1,12,13,4,17,23,spec1d.vdgc1.012.gcA17.fits.gz,--,--
---,GCA,Y,VCC1075,187.03674,10.289222,1802.7073,6.052629,4.5716085,6.9834056,1.0,1821.8304,3.3122916,6.584555,1.50770975e-05,9.640003e-06,6.5378563e-06,4.866667e-05,7.06,gcA19,vdgc1,14,15,4,39,46,spec1d.vdgc1.014.gcA19.fits.gz,--,--
---,GCA,Y,VCC1075,187.04433,10.315306,1861.2278,2.5437276,1.4287839,1.2718638,1.0,1875.8278,0.69047767,0.23015922,0.0012339203,0.00022345458,0.00019736988,4.866667e-05,2.93,gcA20,vdgc1,15,16,1,32,36,spec1d.vdgc1.015.gcA20.fits.gz,--,--
---,GCN,Y,VCC1079,187.0423,10.3538885,1323.2156,12.560184,13.214163,11.683929,1.0,1333.0521,12.498647,9.956888,-1.5877651e-05,1.3676128e-05,1.9947132e-05,4.866667e-05,6.54,gcN120,vdgc1,26,31,1,122,128,spec1d.vdgc1.026.gcN120.fits.gz,--,--
---,GCN,Y,VCC1079,187.05162,10.367778,1338.6162,13.627155,13.83195,13.110909,1.0,1369.9778,9.316445,7.795393,5.5871988e-05,3.38234e-05,3.4890803e-05,4.866667e-05,6.97,gcN138,vdgc1,27,32,4,32,38,spec1d.vdgc1.027.gcN138.fits.gz,--,--
---,GCN,Y,VCC1079,187.05696,10.373972,1313.909,11.331903,11.525938,10.722818,1.0,1330.3203,7.775379,7.475171,6.03751e-06,2.805274e-05,2.5284158e-05,4.866667e-05,4.94,gcN145,vdgc1,29,34,4,20,26,spec1d.vdgc1.029.gcN145.fits.gz,--,--
---,GCN,Y,VCC1079,187.05429,10.377334,1358.58,6.8130116,8.182219,4.8669662,1.0,1376.2621,7.945497,4.2029076,1.0273774e-05,5.0034614e-06,7.0382025e-06,4.866667e-05,7.02,gcN147,vdgc1,30,35,4,30,36,spec1d.vdgc1.030.gcN147.fits.gz,--,--
---,GCN,Y,VCC1076,187.0515,10.524445,1650.4119,11.4403515,11.538051,10.929236,1.0,1669.485,10.947573,10.317138,1.4910315e-05,1.1407892e-05,1.12744665e-05,4.866667e-05,6.99,gcN206,vdgc1,37,44,4,114,120,spec1d.vdgc1.037.gcN206.fits.gz,--,--


In [ ]:
import csv
test1 = []
with open('catalog.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        test1.append(row)